# A Demonstration of the Diffie&ndash;Hellman Key Agreement Protocol
The Diffie&ndash;Hellman protocol presented below demonstrates a confidential message exchange between two parties&mdash;Alice and Bob&mdash;over an insecure channel. Further, a third party&mdash;Eve&mdash;who intercepts the message encrypted using this protocol, has no feasible way of learning its content.

Diffie and Hellman first introduced this protocol in 1976, in their seminal paper [*New Directions in Cryptography*](https://ee.stanford.edu/~hellman/publications/24.pdf). In it, the authors imagined a novel cryptosystem in which a publicly&ndash;known key could be used to encrypt a message that only the holder of a private key could decrypt, thereby solving a problem that bedeviled all previously known ciphers; that of exchanging encryption keys *securely* between trusted parties. Such a *public&ndash;key* cryptosystem could further be employed to guarantee message authenticity (a property that encryption alone could not), via a process they called *digital signature*.

Although Diffie and Hellman fell short of presenting a practical implementation for such a cryptosystem, they laid the critical groundwork for it (it would fall to Rivest, Shamir and Adlemen to flesh it out in their 1978 paper [*A Method for Obtaining Digital Signatures and Public&ndash;Key Cryptosystems*](https://people.csail.mit.edu/rivest/Rsapaper.pdf)).

Diffie and Hellman did, however, leave us with a very important, practical contribution; a protocol for secure key&ndash;exchange that remains to this day the de facto method of exchanging keys over insecure channels. The method is based on a one&ndash;way function that pits the difficulty of solving the *discrete logarithm problem* (or *DLP*) in large&ndash;order [multiplicative integer groups](https://en.wikipedia.org/wiki/Multiplicative_group) against the ease of multiplying integers in such groups.

This protocol is demonstrated in the sections that follow.

## The Source Code
The following sequence of function calls to the `dh` module (located in the [/src](https://github.com/dchampion/crypto/tree/master/code/src) directory of this repository) illustrates a session in which Alice encrypts a private message, and then transmits it to Bob over an insecure channel. Meanwhile, Eve&mdash;a passive adversary&mdash;intercepts the message transmitted on the insecure channel.

It is assumed that Alice, Bob and Eve all possess their own copy of the `dh` module; but that only Alice and Bob possess their own private keys, and also a shared encryption key each generates using their private keys as input. Eve, not having access to either Alice's or Bob's private keys, therefore cannot feasibly derive the session key Alice uses to encrypt the message she sends to Bob, thereby preventing Eve from learning the contents of the message.

Values below surrounded in square brackets [ ] are public (i.e., they can be transmitted over the insecure channel with no compromise to the security of the system, even if they are observed by Eve), and those that are not in square brackets are private (i.e., they must not be transmitted over the insecure channel).

Explanations of behavior appear below the code snippets and their outputs.

In [1]:
import sys
sys.path.append("../code/src")
import dh
sys.path.append("../code/test")
import sym

Import the `dh` module, and a module `sym` that implements an elementary symmetric cipher that Alice and Bob will use in the protocol.

In [2]:
q, p, g = dh.generate_parameters(dh.min_p_bit_len)

The protocol begins with Alice computing the domain parameters she and Bob will use to set up a session in which Alice can safely transmit an encrypted message to Bob.

The argument `dh.min_p_bit_len` passed to the function `dh.generate_parameters` specifies the size, in bits, of the modulus to be used in the protocol; the security of the system is directly proportional to the size of this modulus (`dh.min_p_bit_len` is an integer constant whose value is `2048`).

The parameter *[p]* returned by `dh.generate_parameters` is the modulus (a prime number). *[q]* (also a prime number) is the size of the smallest subgroup of the full group modulo *[p]*; the subgroup within which the public keys (to be generated in a subsequent step) must fall. And *[g]* is the generator of this subgroup.

All three parameters returned by this function are public, and may be observed by Eve with no compormise to the security of the system.

In [3]:
print(p.bit_length())
print(q.bit_length())

2048
256


Here we print the bit lengths of the first two parameters returned by `dh.generate_parameters`. Note that the length of *[p]*, the  modulus, is in fact the `2048` bits requested. The bit length of *[q]* (`256`) represents the size of smallest subgroup of the full group modulo *[p]*.

We are not particularly interested in the bit&ndash;length of the generator *[g]*, but rather that it will generate the entire subgroup represented by *[q]*. We prove this by calling the function `dh._validate_parameters`.

In [4]:
dh.validate_parameters(q, p, g)

If this function does not a raise an exception, we know the parameters are suitable to generate keys to be used for a secure message exchange.

In [5]:
kA, KA = dh.generate_keypair(q, p, g)

Alice now computes a pair of keys; *kA* is her private key, and *[KA]* her public key (the keys are suffixed with the captital letter *A* to indicate that they belong to Alice).

Alice transmits the domain parameters *[p]*, *[q]*, *[g]*, and her public key *[KA]*, to Bob.

Then Bob calls the same function Alice did above to validate the domain parameters, and another function `dh.validate_pub_key` to validate Alice's public key *[KA]*.

In [6]:
dh.validate_parameters(q, p, g)
dh.validate_pub_key(KA, q, p)

If neither function raises an exception, Bob knows that the domain parameters and Alice's public key are all valid, and can therefore safely proceed with the protocol.

In [7]:
kB, KB = dh.generate_keypair(q, p, g)

Using the domain parameters Alice sent him, Bob now computes a key pair of his own (storing them in the variables *kB* and *[KB]*).

Bob transmits his public key *[KB]* to Alice.

In [8]:
dh.validate_pub_key(KB, q, p)

On receiving Bob's public key *[KB]*, Alice must validate it in the same way Bob validated Alice's public key.

In [9]:
kSessionA = dh.generate_session_key(KB, kA, p)

Now Alice computes the session key that will be used to encrypt the message she will transmit to Bob.

To do this, she calls the function `dh.generate_session_key`, passing it Bob's public key *[KB]*, Alice's private key *kA* and the modulus *[p]*.

Basically, all this function does is raise Bob's public key *[KB]* to the power of Alice's private key *kA*, and take the result modulo *[p]*.

In [10]:
kSessionB = dh.generate_session_key(KA, kB, p)

Bob calls the same function Alice did to compute a session key of his own, only in Bob's case he passes Alice's public key *[KA]* and his private key *kB* to `dh.generate_session_key`.

As in Alice's case, the function raises Alice's public key *[KA]* to the power of Bob's private key *kB*, and takes the result modulo *[p]*.

Herein lies the essential property of DH: *[KA]* raised to the power of *kB* is equal to *[KB]* raised to the power *kA* (both modulo *[p]* of course), thus producing identical session keys *kSessionA* and *kSessionB*.

Where does this leave Eve? Having only seen *[KA]* and *[KB]*, Eve cannot from these values alone easily determine the value of the exponents used to compute them (i.e., solve the discrete logarithm problem). This is the *one&ndash;way* function that is the essence of DH.

As with all traditional ciphers, the key Alice uses to encrypt the message is the same one Bob uses to decrypt it. The difference here is that encryption is not predicated on a secure *exchange* of keys; rather, the keys are derived independently by Alice and Bob, and never transmitted over the insecure channel. Because of this, it is perhaps more apt to describe DH as a key *agreement* protocol rather than a key *exchange* protocol.

In [11]:
print(kSessionA == kSessionB)

True


If the essential property of DH holds, both keys&mdash;*kSessionA* and *kSessionB*&mdash;must be equivalent.

In [12]:
mA = "Encrypt me!"

Now, Alice composes a message for Bob, and stores it in the variable *mA*.

In [13]:
mAC = sym.encrypt(kSessionA, mA)

Alice must now encrypt the message. She does this by calling the function `sym.encrypt`, passing it her session key *kSessionA* and the plaintext of her message *mA*. The function returns the ciphertext of *mA*, *[mAC]*.

Alice transmits the ciphertext *[mAC]* to Bob over the insecure channel.

In [14]:
mB = sym.decrypt(kSessionB, mAC)

Bob, having received the ciphertext *[mAC]* from Alice, decrypts it by calling the function `sym.decrypt`, passing it his session key *kSessionB* and the ciphertext *[mAC]*. The result *mB* should be the plaintext of Alice's original message *mA*.

In [15]:
print(mB)

Encrypt me!


Here we print *mB*, proving that Bob has successfully recovered the plaintext of Alice's message.

Finally, Eve, having observed the domain parameters *[p]*, *[q]* and *[g]*; and Alice's and Bob's public keys *[KA]* and *[KB]*, on the insecure channel, has no feasible way of deriving the session key *kSessionA* (or *kSessionB*). Without the session key, Eve cannot decrypt the message.